## Scrapping bantennews.co.id

In [9]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
import re

In [10]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [11]:
def scrape_links(page_number):
    url=f"https://www.bantennews.co.id/category/politik/pemilu-2024/page/{page_number}/"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('div',{"class": "td-block-span6"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [12]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 10 links from page 1
Total Links: 10


In [13]:
def scrape_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "entry-title"})
            if title_elem:
                title_text = title_elem.text.strip()
            else:
                title_text = "Title not found"
            # Author berita
            # author_elem = soup.find('div', {"class": "tags-wrapper"})
            # if author_elem:
            #     author_text = author_elem.find('span')
            #     author_text = author_text.text.split(':')[1]
            # else:
            author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('time', {"class": "entry-date updated td-module-date"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find('li', {"class": "entry-category"})
            if category_elements:
                category_text = category_elements.find('a')
                category_text= category_text.text.strip()
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('div', {"class": "td-post-content td-pb-padding-side"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"
            nama_berita_match = re.search(r'https://www\.(\w+)\.co\.id/', url)
            if nama_berita_match:
                nama_berita = nama_berita_match.group(1)
            else:
                nama_berita = "Nama_berita not found"
            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'nama_berita' : nama_berita,
                            'region':'banten',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [14]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [15]:
df = pd.DataFrame(results)
print("Total Hasil ",len(results))
df.head(10)

Total Hasil  10


,title,author,category,date,content,nama_berita,region,link
0,AHY dan Ridwan Kamil Sudah Dicoret dari Bakal ...,Author not found,Pemilu 2024,"Selasa 19 Sep 2023, 14:12 WIB",JAKARTA – Setelah Partai Demokrat memutuskan u...,bantennews,banten,https://www.bantennews.co.id/ahy-dan-ridwan-ka...
1,"Temui Walikota Serang, Bawaslu Bakal Tertibkan...",Author not found,Pemilu 2024,"Selasa 19 Sep 2023, 00:08 WIB","SERANG – Ketua Bawaslu Kota Serang, Agus Aan b...",bantennews,banten,https://www.bantennews.co.id/temui-walikota-se...
2,Pemungutan Suara di Luar Negeri Diusulkan Dige...,Author not found,Pemilu 2024,"Kamis 21 Sep 2023, 04:13 WIB","SERANG – Ketua KPU RI, Hasyim Asy’ari menyatak...",bantennews,banten,https://www.bantennews.co.id/pemungutan-suara-...
3,"Singgung Etika Politik, Capres-Cawapres Ikut 2...",Author not found,Pemilu 2024,"Selasa 19 Sep 2023, 04:13 WIB",SERANG – Mahkamah Konstitusi menggelar sidang ...,bantennews,banten,https://www.bantennews.co.id/singgung-etika-po...
4,Dukungan Demokrat Jadi Angin Segar untuk Prabo...,Author not found,Pemilu 2024,"Rabu 20 Sep 2023, 06:13 WIB",SERANG – Partai Demokrat telah menentukan arah...,bantennews,banten,https://www.bantennews.co.id/dukungan-demokrat...
5,Sejarawan Bonie Triana Nilai Rano Karno Layak ...,Author not found,Pemilu 2024,"Kamis 21 Sep 2023, 00:11 WIB",SERANG – Sejarawan Bonie Triana menilai sosok ...,bantennews,banten,https://www.bantennews.co.id/sejarawan-bonie-t...
6,"Demi Kemenangan Prabowo, SBY Siap Turun Gunung",Author not found,Pemilu 2024,"Sabtu 23 Sep 2023, 07:08 WIB","SERANG – Ketua Umum DPP Partai Gerindra, Prabo...",bantennews,banten,https://www.bantennews.co.id/demi-kemenangan-p...
7,Dinilai Bisa Menang Banyak Jika Ganjar-Prabowo...,Author not found,Pemilu 2024,"Sabtu 23 Sep 2023, 03:16 WIB",SERANG – Analis Politik dari Universitas Andal...,bantennews,banten,https://www.bantennews.co.id/dinilai-bisa-mena...
8,"Diwacanakan Lagi Duet dengan Ganjar, Begini Ka...",Author not found,Pemilu 2024,"Jumat 22 Sep 2023, 09:12 WIB",JAKARTA – Ketua Umum Partai Gerindra Prabowo S...,bantennews,banten,https://www.bantennews.co.id/diwacanakan-lagi-...
9,Tokoh Lintas Agama di Cilegon Diharapkan Ikut ...,Author not found,Pemilu 2024,"Selasa 19 Sep 2023, 02:17 WIB",CILEGON – Guna mewujudkan Kota Cilegon tetap d...,bantennews,banten,https://www.bantennews.co.id/tokoh-lintas-agam...


In [16]:
# current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
# excel_file_name = f'../../tempat_hasil_daerah/banten_antranews_{current_datetime}.xlsx'
# df.to_excel(excel_file_name, index=False)

# print(f'Data has been saved to {excel_file_name}')